## Cubic Spline Interpolation

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
Ia, Ib = -1.0, 1.0                 ### Interval endpoints
N = 4                              ### Number of sub-intervals
xi = np.linspace(Ia,Ib,N+1)        ### Uniform sample locs  

### Functions to test - and 2nd derivatives

def f_sin(x): return np.sin(3*x)         ### Sine wave  
def f_runge(x): return 1./(1+(4.*x)**2)  ### Runge fn
def f_step(x): return np.array(x > 0., dtype=np.int)        ### Heaviside fn
def f_kink(x): return np.abs(x)          ### Kink fn


In [ ]:
def S(x, xj, a, b, c, d):
    return a + b * (x-xj) + c * (x-xj)**2 + d * (x-xj)**3

def rhs(grid, fi):           ### RHS - interior points
    N = len(grid)-1
    h = grid[1:] - grid[:-1]   # width of interval i - vector size N
    rhs = np.zeros(N+1)
    for i in range(1, N):
        rhs[i] = 3.*(fi[i+1] - fi[i])/h[i] - 3.*(fi[i] - fi[i-1])/h[i-1]
    return rhs

def system_matrix(grid):
    N = len(grid)-1
    h = grid[1:] - grid[:-1]   # width of interval i - vector size N
    A = np.zeros((N+1,N+1))
    ### Equations describing conditions at interior nodes
    for i in range(1, N):
        A[i,i]   = 2*(h[i-1]+h[i])
        A[i,i-1] = h[i-1]
        A[i,i+1] = h[i]
    return A

def spline_natural(xi, fi, xx):
    """
    One-shot function for spline interpolation (with natural BCs).
    
    Args:
      xi (array, n+1): Sample locations
      fi (array, n+1): Sample values
      xx (array, M):   Reconstuction locations
    Return:
      ff (array, M): Reconstructed values at xx
    """
    h = xi[1:] - xi[:-1]       # Interval width
    N = len(h)
                               ### Setup system
    A = system_matrix(xi)      # Left-hand side 
    frhs = rhs(xi, fi)         # Right-hand side
    A[0,0] = A[N,N] = 1        # BC for LHS (natural)
    frhs[0] = 0                # BC for RHS (natural)
    frhs[-1] = 0 
                               ### Solve system for coefficients
    c = np.linalg.solve(A, frhs)
    a = fi[:]                                                  # N+1
    b = (a[1:] - a[:-1]) / h[:] - h[:]/3. * (2*c[:-1] + c[1:]) # N
    d = (c[1:] - c[:-1]) / (3. * h[:])                         # N
                               ### Reconstuct spline at locations xx  
    ii = np.digitize(xx, xi)   # Find to which interval each xx belongs
    ii = np.fmin(np.fmax(ii-1,0),N-1) 
    ff = np.zeros(xx.shape)
    for j, i in enumerate(ii): # Compute spline for each x
        ff[j] = S(xx[j], xi[i], a[i], b[i], c[i], d[i])
    return ff

In [ ]:
#### TODO - Question (a) - Change N and see how splines behave.
N = 19                               ### Number of sub-intervals
f = f_kink # f_sin, f_runge, f_step, f_kink
xx = np.linspace(Ia,Ib,201)            ### Uniform reconstruction locations
xi = np.linspace(Ia,Ib,N+1)            ### Uniform nodes
#xi = np.random.random(N-1)*(Ib-Ia)+Ia  ### Random nodes
#xi = np.hstack(([Ia], xi, [Ib]))        
xi.sort()
fi = f(xi)                             ### Sample values 

plt.plot(xx, spline_natural(xi, fi, xx), '-b')
plt.plot(xx, f(xx), '-g')
plt.plot(xi, f(xi), 'ok')